In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data = data.drop(['RowNumber' , 'CustomerId' , 'Surname'],axis =1)

In [4]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [6]:
data.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1
6,822,France,1,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,0,29,4,115046.74,4,1,0,119346.88,1
8,501,France,1,44,4,142051.07,2,0,1,74940.50,0
9,684,France,1,27,2,134603.88,1,1,1,71725.73,0


In [7]:
from sklearn.preprocessing import OneHotEncoder
onehot_Geography = OneHotEncoder()
temp = onehot_Geography.fit_transform(data[['Geography']]).toarray()

In [8]:
temp_df = pd.DataFrame(temp , columns=onehot_Geography.get_feature_names_out(['Geography']))

In [9]:
temp_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
data = pd.concat([data.drop(columns=['Geography']), temp_df], axis=1)

In [11]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [12]:
## Save the encoders and sscaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_Geography.pkl','wb') as file:
    pickle.dump(onehot_Geography,file)

In [13]:
X=data.drop('Exited',axis=1)
y=data['Exited']
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2 , random_state=42)

In [14]:
print(x_train.shape)
print(x_test.shape)

(8000, 12)
(2000, 12)


In [15]:
standard_scalar = StandardScaler()
x_train = standard_scalar.fit_transform(x_train)
x_test = standard_scalar.transform(x_test)

In [16]:
with open('standard_scalar.pkl' ,'wb') as file:
    pickle.dump(standard_scalar , file)

### ANN


In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard


In [18]:
model = Sequential([
    Dense(64 , activation = 'relu' , input_shape= (x_train.shape[1] ,)),
    Dense(32 , activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [21]:
model.compile(optimizer=opt , loss=loss , metrics=['accuracy'])

In [22]:
from datetime import datetime

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping = EarlyStopping(monitor='val_loss' ,patience=10 , restore_best_weights=True)


In [23]:
final_model = model.fit(
    x_train , y_train , validation_data=(x_test , y_test) , epochs = 100,
    callbacks = [tensorflow_callbacks , early_stopping]
)

Epoch 1/100


250/250 [==============================] - 3s 5ms/step - loss: 0.3983 - accuracy: 0.8364 - val_loss: 0.3604 - val_accuracy: 0.8540
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3548 - accuracy: 0.8539 - val_loss: 0.3535 - val_accuracy: 0.8540
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3494 - accuracy: 0.8540 - val_loss: 0.3654 - val_accuracy: 0.8445
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3442 - accuracy: 0.8584 - val_loss: 0.3415 - val_accuracy: 0.8585
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3400 - accuracy: 0.8604 - val_loss: 0.3443 - val_accuracy: 0.8625
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3375 - accuracy: 0.8579 - val_loss: 0.3517 - val_accuracy: 0.8525
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3376 - accuracy: 0.8625 - val_loss: 0.3437 - val_accuracy: 0.85

In [24]:
model.save('model.h5')

d:\CHURN MODEL\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
%load_ext tensorboard

In [35]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 24700), started 0:00:09 ago. (Use '!kill 24700' to kill it.)